# Hands-on

## Setup

In [ ]:
# ffmpeg - Working with videos, audios, and streaming media files
!sudo apt install -y ffmpeg
# mediapy - dev dependency; displays videos/images in the notebook
!pip install -q mediapy
!pip uninstall -q -y opencv-python-headless
!pip install -q "opencv-python-headless<4.3"

In [3]:
# Import libraries
import pathlib

import matplotlib as mpl
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
import PIL

import tensorflow as tf
import tensorflow_hub as hub
import tqdm

mpl.rcParams.update({
    'font.size': 10,
})

In [4]:
# Getting the kinetics 600 label
# Printing the first few labels

labels_path = tf.keras.utils.get_file(
    fname="labels.txt",
    origin="https://raw.githubusercontent.com/tensorflow/models/f8af2291cced43fc9f1d9b41ddbf772ae7b0d7d2/official/projects/movinet/files/kinetics_600_labels.txt"
)
labels_path = pathlib.Path(labels_path)

lines = labels_path.read_text().splitlines()
KINETICS_600_LABELS = np.array([line.strip() for line in lines])
KINETICS_600_LABELS[:20]

9209/9209 [==============================] - 0s 0us/step


array(['abseiling', 'acting in play', 'adjusting glasses', 'air drumming',
       'alligator wrestling', 'answering questions', 'applauding',
       'applying cream', 'archaeological excavation', 'archery',
       'arguing', 'arm wrestling', 'arranging flowers',
       'assembling bicycle', 'assembling computer',
       'attending conference', 'auctioning', 'backflip (human)',
       'baking cookies', 'bandaging'], dtype='<U49')

In [23]:
cartwheel_url = 'https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExZHoxc3JlZjA3bTg0aTZzNzBmdnk0enNpdm1haTVxajZmMDRpNGN4cyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/IhsEJ88BJ3MQl9UAoZ/giphy.gif'
cartwheel_path = tf.keras.utils.get_file(
    fname='cartwheel.gif',
    origin=cartwheel_url,
    cache_dir='.',
    cache_subdir='.',
)

2280185/2280185 [==============================] - 0s 0us/step


In [24]:
# @title Loading a gif
def load_gif(file_path, image_size=(224, 224)):
    # Load a gif file, convert it to a TF tensor
    raw = tf.io.read_file(file_path)
    video = tf.io.decode_gif(raw)
    # Resize the video
    video = tf.image.resize(video, image_size)
    # change dtype to a float32
    # Hub models always want images ormalized to [0, 1]
    video = tf.cast(video, tf.float32) / 255
    return video

In [25]:
cartwheel = load_gif(cartwheel_path)
cartwheel.shape

TensorShape([76, 224, 224, 3])

## How to use the model

### Base model

In [26]:
%%time
id = 'a2'
mode = 'base'
version = '3'
hub_url = f'https://tfhub.dev/tensorflow/movinet/{id}/{mode}/kinetics-600/classification/{version}'
model = hub.load(hub_url)

CPU times: user 21.1 s, sys: 1.45 s, total: 22.5 s
Wall time: 24.1 s


In [ ]:
sig = model.signatures['serving_default']
print(sig.pretty_printed_signature())

In [38]:
# Adding the outer batch dimension to the video
# warmup
sig(image = cartwheel[tf.newaxis, :1])

{'classifier_head': <tf.Tensor: shape=(1, 600), dtype=float32, numpy=
 array([[ 4.10832852e-01, -1.75833368e+00,  7.21597135e-01,
         -1.99860299e+00, -3.28061581e+00,  2.11836314e+00,
          4.08527613e+00,  2.53223634e+00,  2.31717277e+00,
         -1.90893590e+00,  1.77989674e+00,  1.36765051e+00,
         -1.55218279e+00,  6.24242961e-01, -1.41566801e+00,
          1.99393201e+00,  1.92371225e+00, -5.23871899e-01,
         -4.85216141e-01, -8.07527125e-01, -1.82610130e+00,
         -4.09451008e+00,  1.52345586e+00, -1.96385741e-01,
          2.55342931e-01, -3.34995836e-01, -1.60672709e-01,
         -4.49170637e+00,  1.37373209e+00, -3.67361474e+00,
          1.58103204e+00, -2.12662384e-01, -6.87839329e-01,
          1.62189794e+00, -1.63132775e+00, -2.17364120e+00,
          3.14513493e+00, -7.07009315e-01,  1.70156753e+00,
         -2.57661968e-01, -2.05417585e+00,  3.04886603e+00,
         -1.49553049e+00, -1.81551540e+00, -1.62883198e+00,
         -1.95249343e+00, -1.8